In [1]:
import re
import tensorflow as tf
import numpy as np
import time
import os
import collections
import random
import nltk
import time
import pickle

In [2]:
def clearstring(string):
    string = re.sub('[^A-Za-z\- ]+', '', string)
    string = string.split(' ')
    string = filter(None, string)
    string = [y.strip() for y in string]
    return [y.lower() for y in string if len(y) > 3 and y.find('nbsp') < 0]

def build_dataset(words, n_words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

data_index = 0

# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    global data
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
        context_words = [w for w in range(span) if w != skip_window]
        words_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
        if data_index == len(data):
            for word in data[:span]:
                buffer.append(word)
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [3]:
with open('books/The-Hunger-Games.txt', 'r',encoding = "ISO-8859-1") as fopen:
    vocabulary = clearstring(' '.join(nltk.word_tokenize(fopen.read())))
print('example 10 words:',vocabulary[:10])
print('size corpus:',len(vocabulary))
vocabulary_size = len(list(set(vocabulary)))
print('size of unique words:',vocabulary_size)
dimension = 128
skip_window = 1
num_skips = 2
batch_size = 64
location = os.getcwd()

example 10 words: ['hunger', 'games', 'hunger', 'games', 'suzanne', 'collins', 'part', 'tributes', 'when', 'wake']
size corpus: 54070
size of unique words: 7390


In [4]:
data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,vocabulary_size)
del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Most common words (+UNK) [['UNK', 1], ('that', 831), ('with', 634), ('have', 596), ('from', 526)]
Sample data [241, 75, 241, 75, 5203, 5579, 238, 56, 9, 602] ['hunger', 'games', 'hunger', 'games', 'suzanne', 'collins', 'part', 'tributes', 'when', 'wake']


In [5]:
valid_size = 16
valid_window = 100
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

In [6]:
graph = tf.Graph()

with graph.as_default():

    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
        embeddings = tf.Variable(tf.random_uniform([vocabulary_size, dimension], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)
        nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, dimension],
                                                      stddev=1.0 / np.sqrt(dimension)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
        loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights,
                                             biases=nce_biases,
                                             labels=train_labels,
                                             inputs=embed,
                                             num_sampled=batch_size / 2,
                                             num_classes=vocabulary_size))
        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

        # Compute the cosine similarity between minibatch examples and all embeddings.
        norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
        normalized_embeddings = embeddings / norm
        valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
        similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
        init = tf.global_variables_initializer()

num_steps = 200000

In [7]:
with tf.Session(graph=graph) as session:
    init.run()
    print('Initialized')

    average_loss = 0
    for step in range(num_steps):
        batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

        if step % 10000 == 0:
            sim = similarity.eval()
            for i in range(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % valid_word
                for k in range(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = '%s %s,' % (log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  120.172103882
Nearest to UNK: nightmare, carve, desperation, strategy, weeks, buried, cloak, vines,
Nearest to never: disaster, explanations, undetectable, deadliest-looking, skys, claws, canister, forward,
Nearest to more: frogs, cleaned, rebounds, forty, crosses, shrunk, peacekeeper, fighter,
Nearest to cato: pitied, aloud, nightfall, circumstances, breached, suspect, convulses, catching,
Nearest to only: boxing, labs, heavily, fate, reserved, flops, maintain, crusts,
Nearest to dont: interaction, reach, wept, slimy, insult, good-natured, flounder, babble,
Nearest to back: clenched, rabid, president, dangerous, insufficient, mean, forest, entitled,
Nearest to take: blinding, kick, daily, ahead, drenched, badly, shaped, away,
Nearest to moment: amount, grain, gulp, retire, slashes, massage, ration, notices,
Nearest to capitol: naked, reenactments, tumble, affection, peering, dragging, hazel, twitch,
Nearest to like: oblivious, hobbled, actions, m

Average loss at step  44000 :  2.48844221389
Average loss at step  46000 :  2.44541312569
Average loss at step  48000 :  2.42632800174
Average loss at step  50000 :  2.40803596598
Nearest to UNK: nightmare, rabid, carve, liking, bottom, bowls, vines, thefire,
Nearest to never: couldve, ignored, explanations, remake, disaster, strikes, puzzled, palms,
Nearest to more: fighter, frogs, strolling, uncomplaining, disappeared, wears, yard, personally,
Nearest to cato: west, razor-sharp, personal, halfway, groggy, unscathed, overcome, dunks,
Nearest to only: boxing, condenses, contestants, discover, pursue, herbs, bought, complimentary,
Nearest to dont: good-natured, wept, lunatic, babble, looking, tomorrows, winners, glue,
Nearest to back: nice, indifferent, party, unprecedented, effective, underarms, envied, maces,
Nearest to take: prop, drenched, interested, boiled, securing, splashing, arrived, plunge,
Nearest to moment: fare, gulp, thread, trusting, shifted, ration, chain, destinations,


Average loss at step  92000 :  2.13884354791
Average loss at step  94000 :  2.12831764519
Average loss at step  96000 :  2.13333292583
Average loss at step  98000 :  2.13385108411
Average loss at step  100000 :  2.09725078881
Nearest to UNK: nightmare, liking, rabid, carve, cloak, thefire, source, bowls,
Nearest to never: couldve, puzzled, spirits, ignored, strikes, gorgeous-looking, disaster, contrived,
Nearest to more: fighter, frogs, personally, uncomplaining, rethought, strolling, adult, revoked,
Nearest to cato: west, advises, halfway, razor-sharp, luck, personal, inhale, expectations,
Nearest to only: boxing, complimentary, pursue, enlist, manufactured, warehouse, discover, uncommon,
Nearest to dont: lunatic, babble, forthcoming, winners, orders, good-natured, mean, opinion,
Nearest to back: party, nice, underarms, envied, woes, outsmarted, unprecedented, effective,
Nearest to take: drenched, demean, prop, interested, splashing, plunk, plunge, boiled,
Nearest to moment: fare, tru

Average loss at step  142000 :  2.06525138831
Average loss at step  144000 :  2.04742107794
Average loss at step  146000 :  2.05790014818
Average loss at step  148000 :  2.05451809692
Average loss at step  150000 :  2.05719515824
Nearest to UNK: nightmare, rabid, liking, carve, cloak, thefire, source, pond,
Nearest to never: couldve, puzzled, contrived, caught, palms, spirits, stabbing, easier,
Nearest to more: fighter, frogs, rethought, personally, adult, uncomplaining, revoked, strolling,
Nearest to cato: west, overhear, personal, luck, advises, expectations, evils, groggy,
Nearest to only: pursue, complimentary, boxing, expected, herbs, require, contestants, uncommon,
Nearest to dont: lunatic, good-natured, forthcoming, wept, dictate, babble, plaintively, winners,
Nearest to back: underarms, hysterically, envied, woes, bolt, party, nice, unprecedented,
Nearest to take: demean, drenched, interested, plunk, boiled, splashing, accumulated, prop,
Nearest to moment: fare, delivered, remi

Average loss at step  192000 :  2.02116066307
Average loss at step  194000 :  2.04561023101
Average loss at step  196000 :  2.04172439814
Average loss at step  198000 :  2.0058203586


In [8]:
with open('hungergame-list.p', 'wb') as fopen:
    pickle.dump(list(reverse_dictionary.values()), fopen)

In [9]:
with open('hungergame-vector.p', 'wb') as fopen:
    pickle.dump(final_embeddings, fopen)